<a href="https://colab.research.google.com/github/ab2gbl/Master2/blob/main/ITFFC/TP/ITFFC_first_80k_4ep_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!git clone https://github.com/liamdugan/human-detection.git

In [1]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from nltk.tokenize import sent_tokenize
import pandas as pd
import numpy as np

In [2]:
!pip install opendatasets

# ROFT dataset

In [3]:
!cp /content/drive/MyDrive/Master2/ITFFC/roft.csv ./roft.csv

In [24]:
import pandas as pd

# Load the RoFT dataset
roft_df = pd.read_csv("roft.csv")

# Inspect the first few rows
print(roft_df.head())

# Check for missing values
print(roft_df.isnull().sum())

                               date      model  dataset  annotator group  \
0  2021-08-31 17:11:39.095000+00:00  finetuned  Recipes       1666     A   
1  2021-09-06 21:54:48.912000+00:00  finetuned  Recipes       1666     A   
2  2021-09-06 21:55:07.069000+00:00  finetuned  Recipes       1666     A   
3  2021-09-06 21:58:44.944000+00:00  finetuned  Recipes       1666     A   
4  2021-09-06 21:59:16.230000+00:00  finetuned  Recipes       1666     A   

   dec_strat_value  predicted_boundary_index  true_boundary_index  points  \
0              0.4                         0                    2       0   
1              0.4                         8                    8       5   
2              0.4                         0                    7       0   
3              0.4                         1                    7       0   
4              0.4                         1                    2       0   

                    reason  prompt  \
0  ['9123971792800820313']   13803   
1   

In [25]:
# Drop rows where `gen_body` is NaN
roft_df = roft_df.dropna(subset=["gen_body"])

# Create a binary label column: AI-generated (1) or Human-written (0)
roft_df["label"] = roft_df["model"].apply(lambda x: 1 if x != "baseline" else 0)

# Select only the relevant columns
roft_df = roft_df[["gen_body", "label"]]

roft_df.rename(columns={'gen_body': 'text'}, inplace=True)

<ipython-input-25-bf88d409cc97>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  roft_df["label"] = roft_df["model"].apply(lambda x: 1 if x != "baseline" else 0)


In [6]:
num_rows_label_0 = roft_df[roft_df["label"] == 0].shape[0]
num_rows_label_1 = roft_df[roft_df["label"] == 1].shape[0]


print(f"Number of rows with label=0: {num_rows_label_0}")
print(f"Number of rows with label=1: {num_rows_label_1}")

Number of rows with label=0: 196
Number of rows with label=1: 24021


# Augmented data for LLM - Detect AI Generated Text

In [7]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/jdragonxherrera/augmented-data-for-llm-detect-ai-generated-text")
# 9aba4703ac130612b1bc4c2d83ed8627
#https://www.kaggle.com/datasets/ratthachat/writing-prompts

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: abdessamiguebli
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/jdragonxherrera/augmented-data-for-llm-detect-ai-generated-text


100%|██████████| 314M/314M [00:03<00:00, 100MB/s]


In [11]:
aug_train = pd.read_csv('/content/augmented-data-for-llm-detect-ai-generated-text/final_train.csv')
aug_test = pd.read_csv('/content/augmented-data-for-llm-detect-ai-generated-text/final_test.csv')


In [12]:
aug_train.columns

Index(['text', 'label'], dtype='object')

In [13]:
aug_df = pd.concat([aug_train, aug_test], ignore_index=True)

In [15]:
aug_df.count()

,0
text,433564
label,433564


In [18]:
aug_df['label'] = aug_df['label'].astype(int)

In [21]:
aug_df = aug_df.dropna(how='any')

In [26]:
aug_df.drop_duplicates(inplace=True)

In [27]:
aug_df['label'].value_counts()

,count
label,
0,277981
1,155565


# writing prompts dataset


In [28]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/ratthachat/writing-prompts")
# 9aba4703ac130612b1bc4c2d83ed8627
#https://www.kaggle.com/datasets/ratthachat/writing-prompts

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: abdessamiguebli
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/ratthachat/writing-prompts


100%|██████████| 370M/370M [00:06<00:00, 62.6MB/s]


In [29]:
# Load the prompts and stories
with open("./writing-prompts/writingPrompts/train.wp_source", "r") as f:
    prompts = f.readlines()

with open("./writing-prompts/writingPrompts/train.wp_target", "r") as f:
    stories = f.readlines()

print(f"Number of prompts: {len(prompts)}")
print(f"Number of stories: {len(stories)}")

Number of prompts: 272600
Number of stories: 272600


In [31]:
import pandas as pd

# Create a DataFrame
human_data = pd.DataFrame({
    "text": stories,  # Use stories as human-generated text
    "label": 0        # Human-generated
})

# Merge datasets

In [32]:
# Assuming your RoFT dataset is in a DataFrame `roft_data`
df = pd.concat([roft_df, aug_df, human_data], ignore_index=True)

In [34]:
df = aug_df.dropna(how='any')
df.drop_duplicates(inplace=True)

In [35]:
print(df['label'].value_counts())
print("Columns in balanced_data:")
print(df.columns)

label
0    277981
1    155565
Name: count, dtype: int64
Columns in balanced_data:
Index(['text', 'label'], dtype='object')


In [36]:
df.to_csv('AiVsHuman.csv')

In [37]:
#!cp ./AiVsHuman.csv /content/drive/MyDrive/Master2/ITFFC/AiVsHuman.csv

# model trainning

In [2]:
!cp /content/drive/MyDrive/Master2/ITFFC/AiVsHuman.csv ./AiVsHuman.csv

In [29]:
df = pd.read_csv("./AiVsHuman.csv")
print(df['label'].value_counts())

label
0    277981
1    155565
Name: count, dtype: int64


In [30]:
# Separate the dataset by label
df_label_0 = df[df['label'] == 0]
df_label_1 = df[df['label'] == 1]

# Sample 80,000 rows from each label (if there are enough rows)
df_label_0_sampled = df_label_0.sample(n=40000, random_state=42)
df_label_1_sampled = df_label_1.sample(n=40000, random_state=42)

# Concatenate the sampled data back together
df_balanced = pd.concat([df_label_0_sampled, df_label_1_sampled])

# Shuffle the resulting dataset
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Check the new label distribution
print(df_balanced['label'].value_counts())

label
1    40000
0    40000
Name: count, dtype: int64


In [31]:
df = df_balanced

In [32]:
import re

def clean_text(text):
    text = text.lower()  # Lowercase
    text = re.sub(r"\s*sep\s*", " ", text)  # Remove SEP tokens
    text = re.sub(r"\s+", " ", text).strip()  # Normalize spaces
    return text

df["text"] = df["text"].apply(clean_text)

In [ ]:
'''from sklearn.utils import shuffle

# Shuffle the dataset
df = shuffle(df, random_state=42)'''


'from sklearn.utils import shuffle\n\n# Shuffle the dataset\ndf = shuffle(df, random_state=42)'

In [33]:
from sklearn.model_selection import train_test_split

# Split into train (80%), validation (10%), and test (10%)
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["text"], df["label"], test_size=0.2, random_state=42
)

val_texts, test_texts, val_labels, test_labels = train_test_split(
    test_texts, test_labels, test_size=0.5, random_state=42
)

In [21]:
from transformers import RobertaTokenizer

# Load the tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Tokenize the text
# train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=512)
# val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=512)
# test_encodings = tokenizer(list(test_texts), truncation=True, padding=True, max_length=512)

In [34]:
import torch

# Custom Dataset Class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts.iloc[idx]
        label = self.labels.iloc[idx]
        encoding = self.tokenizer(
            text,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(label, dtype=torch.long)
        }
# Create datasets
train_dataset = CustomDataset(train_texts, train_labels , tokenizer, max_length=512)
val_dataset = CustomDataset(val_texts, val_labels , tokenizer, max_length=512)
test_dataset = CustomDataset( test_texts, test_labels , tokenizer, max_length=512)
# test_dataset = CustomDataset(test_encodings, test_labels.tolist())

In [35]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [36]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

In [39]:
from transformers import RobertaForSequenceClassification, Trainer, TrainingArguments

# Load the pre-trained RoBERTa model
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Evaluate at the end of each epoch
    save_strategy="epoch",        # Save model at the end of each epoch
    learning_rate=3e-5,           # Slightly higher learning rate for faster convergence
    per_device_train_batch_size=8,  # Reduce batch size to fit GPU memory
    per_device_eval_batch_size=8,   # Match evaluation batch size
    num_train_epochs=4,           # Reduce the number of epochs
    weight_decay=0.01,            # Regularization
    logging_dir="./logs",         # Directory for logs
    fp16=True,                    # Enable mixed precision for faster training
    gradient_accumulation_steps=2, # Accumulate gradients over 2 steps to simulate a larger batch size
    save_total_limit=1,           # Keep only the most recent model checkpoint
    dataloader_num_workers=4,     # Use multiple workers for data loading

)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics  # Add metrics
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [40]:

# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.021500,0.010861,0.998125,0.998289,0.998045,0.998167
2,0.009200,0.012797,0.996875,0.994408,0.999511,0.996953
3,0.005800,0.015149,0.997375,0.995376,0.999511,0.997439
4,0.001300,0.012787,0.998375,0.997075,0.999756,0.998414


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: T

TrainOutput(global_step=16000, training_loss=0.01853624709136784, metrics={'train_runtime': 7249.403, 'train_samples_per_second': 35.313, 'train_steps_per_second': 2.207, 'total_flos': 6.735643017216e+16, 'train_loss': 0.01853624709136784, 'epoch': 4.0})

In [41]:
# Save the model
model.save_pretrained("/content/drive/MyDrive/Master2/ITFFC/first_80k_4ep_model")

# Save the tokenizer
tokenizer.save_pretrained("/content/drive/MyDrive/Master2/ITFFC/first_80k_4ep_model")


('/content/drive/MyDrive/Master2/ITFFC/first_80k_4ep_model/tokenizer_config.json',
 '/content/drive/MyDrive/Master2/ITFFC/first_80k_4ep_model/special_tokens_map.json',
 '/content/drive/MyDrive/Master2/ITFFC/first_80k_4ep_model/vocab.json',
 '/content/drive/MyDrive/Master2/ITFFC/first_80k_4ep_model/merges.txt',
 '/content/drive/MyDrive/Master2/ITFFC/first_80k_4ep_model/added_tokens.json')

In [43]:

metrics = trainer.evaluate()
print("Validation Metrics:", metrics)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Validation Metrics: {'eval_loss': 0.012787416577339172, 'eval_accuracy': 0.998375, 'eval_precision': 0.997075310748233, 'eval_recall': 0.9997556207233627, 'eval_f1': 0.9984136668700427, 'eval_runtime': 72.3077, 'eval_samples_per_second': 110.638, 'eval_steps_per_second': 13.83, 'epoch': 4.0}


In [46]:
from sklearn.metrics import classification_report

predictions = trainer.predict(test_dataset)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [47]:
# Predicted labels
predicted_labels = predictions.predictions.argmax(-1)

# Generate classification report
report = classification_report(test_labels, predicted_labels, target_names=["Human-Generated", "AI-Generated"])
print(report)


                 precision    recall  f1-score   support

Human-Generated       1.00      1.00      1.00      4008
   AI-Generated       1.00      1.00      1.00      3992

       accuracy                           1.00      8000
      macro avg       1.00      1.00      1.00      8000
   weighted avg       1.00      1.00      1.00      8000



# using the model

In [ ]:
# Add explainability (token and sentence level)
def explain(text):
    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)

    # Perform inference with attention outputs
    model.eval()
    with torch.no_grad():
        outputs = model(**inputs, output_attentions=True)
        attentions = outputs.attentions  # Attention scores
        logits = outputs.logits

    # Predicted class
    predicted_class = torch.argmax(logits, dim=1).item()
    labels = ["Human-Generated", "AI-Generated"]

    # Token-level explanation
    attention_scores = attentions[0][0].mean(dim=0).mean(dim=0)  # Average over heads
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    token_attention = list(zip(tokens, attention_scores.tolist()))

    # Sentence-level explanation
    sentences = sent_tokenize(text)
    sentence_scores = []
    for sentence in sentences:
        sentence_inputs = tokenizer(sentence, return_tensors="pt", truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**sentence_inputs, output_attentions=True)
            attentions = outputs.attentions[0]
            attention_scores = attentions.mean(dim=1).mean(dim=0)
            sentence_scores.append(attention_scores.mean().item())

    return {
        "predicted_label": labels[predicted_class],
        "token_explanations": token_attention,
        "sentence_explanations": list(zip(sentences, sentence_scores))
    }


In [ ]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer

model_path = "/content/drive/MyDrive/Master2/ITFFC/third_model"
model = RobertaForSequenceClassification.from_pretrained(model_path)

tokenizer = RobertaTokenizer.from_pretrained(model_path)

In [ ]:
# Set the model to evaluation mode
model.eval()

# Prepare the test dataset
texts = test_dataset["text"]  # Adjust key based on your test dataset
true_labels = test_dataset["label"]

# Tokenize the test data
inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")

# Perform inference
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# Get predicted labels
predicted_labels = torch.argmax(logits, dim=1).cpu().numpy()

# Calculate metrics
print(classification_report(true_labels, predicted_labels))
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")


In [ ]:
import torch

# Set the device (use GPU if available, else fallback to CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the appropriate device
model = model.to(device)

# Input text
text = """
As the sun dipped below the horizon, painting the sky in hues of orange and pink, the air grew cooler, carrying the faint scent of blooming jasmine. Emma paused on the cobblestone path, letting the beauty of the moment wash over her. It wasn’t often that life slowed down enough to let her notice these small, fleeting wonders.

In the distance, the laughter of children echoed, mingling with the rustle of leaves in the evening breeze. A sense of peace enveloped her, a stark contrast to the chaos of the day. Deadlines, meetings, and the relentless pace of her work had left her drained. But here, in the quiet of this small park, she felt a renewed sense of clarity.

She took a deep breath, her shoulders relaxing. Maybe it wasn’t about finding balance, she thought. Maybe it was about embracing the imbalance, finding moments like these to anchor her, however brief they might be.
"""

# Tokenize the input text
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Move the input tensors to the same device as the model
inputs = {key: value.to(device) for key, value in inputs.items()}

# Perform inference
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

# Get the predicted class
predicted_class = torch.argmax(logits, dim=1).item()

# Interpret the output
labels = ["Human-Generated", "AI-Generated"]
print(f"Predicted label: {labels[predicted_class]}")


Predicted label: Human-Generated
